# Lowlevel Simulation

In [1]:
from Go2Py.sim.mujoco import Go2Sim
import numpy as np

In [ ]:
robot = Go2Sim(mode='lowlevel')

In [ ]:
import matplotlib.pyplot as plt
camera_state = robot.getCameraState()
plt.imshow(camera_state['rgb'])

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d

state = robot.getCameraState(get_pointcloud=True)
pcd = state['pointcloud']

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(pcd[:,0], pcd[:,1], pcd[:,2])
plt.show()


In [ ]:
robot.getJointStates()

In [ ]:
import mujoco
import time
import cv2
camera_dt = 1./60.
robot.standUpReset()
start_time = time.time()
step_counter = 0
while time.time()-start_time < 10:
    state = robot.getJointStates()
    tau = 20*np.eye(12)@(robot.q0 - state['q']).reshape(12,1)
    robot.setCommands(np.zeros(12), np.zeros(12), np.zeros(12), np.zeros(12), tau)
    robot.step()
    if step_counter%(camera_dt//robot.dt)==0:
        camera_state = robot.getCameraState()
        cv2.imshow('rgb', cv2.cvtColor(camera_state['rgb'], cv2.COLOR_BGR2RGB))
        cv2.waitKey(1)
    step_counter+=1
cv2.destroyAllWindows()

# Highlevel Simulation

In [ ]:
from Go2Py.sim.mujoco import Go2Sim
import numpy as np

In [ ]:
robot = Go2Sim(mode='highlevel')
robot.standUpReset()

In [ ]:
import mujoco
import time
import cv2
camera_dt = 1./60.
robot.standUpReset()
start_time = time.time()
step_counter = 0
while time.time()-start_time < 10:
    robot.step(0,0,0., step_height=0,kp=[2, 0.5, 0.5], ki=[0.02, 0.01, 0.01])
    if step_counter%(camera_dt//robot.dt)==0:
        camera_state = robot.getCameraState()
        cv2.imshow('rgb', cv2.cvtColor(camera_state['rgb'], cv2.COLOR_BGR2RGB))
        cv2.waitKey(1)
    step_counter+=1
cv2.destroyAllWindows()

In [ ]:
robot.getLaserScan()

In [ ]:
import matplotlib.pyplot as plt
lidar = robot.getLaserScan(max_range=3.)
idx = np.where(lidar['dist']!=-1)[0]
plt.plot(lidar['dist'][idx])